In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os

In [3]:
def generate_vols(old_r, old_d, w, a, b):
    return np.sqrt(w + a*old_r**2 + b*old_d**2)

In [4]:
N = 100
T = 100

#C = np.eye(N) #uncond_corr_mat
alpha = 0.05
beta = 0.93
w = 0
a = np.ones(N) * 0.05
b = np.ones(N) * 0.9

r_0 = np.zeros(N)
d_0 = np.ones(N)
Q_0 = C
r_series = [r_0] #first return is 0
d_series = [d_0]
Q_series = [Q_0]
H_series = [None]

for t in range(T):
    old_r = r_series[-1] #both vectors of length N
    old_d = d_series[-1]
    s_t_1 = old_r / old_d
    C = Util.cov_to_corr(generate_MTP2(N, 25, sparse_thres = 0.3))
    Q_t = C
    
    #conditional variances at the t-th time step
    d_t = generate_vols(old_r, old_d, w, a, b)
    D_t = np.diag(d_t) 
    
    #Q_t is the conditional covariance matrix, of the devolatized returns
    #R_t is the conditional correlation matrix of the devolatized returns
    #H_t is the conditional covariance matrix of the returns (after factoring in the vols)
    
    Q_t_diag = np.diag(np.power(np.diag(Q_t), -0.5))
    R_t = Q_t_diag.dot(Q_t).dot(Q_t_diag)
    H_t = D_t.dot(R_t).dot(D_t)
    
    r_t = np.random.multivariate_normal(mean = np.zeros(N),
                                        cov = H_t)
    
    r_series.append(r_t)
    d_series.append(d_t)
    Q_series.append(Q_t)
    H_series.append(H_t)

generated C!


KeyboardInterrupt: 

In [7]:
Util.check_MTP2(H_series[2])

(1.0, 0.19837241135297848)

In [8]:
H_series[1]

array([[ 0.9       ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.04109955],
       [ 0.        ,  0.9       ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.9       , ...,  0.        ,
         0.        , -0.05575981],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.9       ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.9       ,  0.        ],
       [-0.04109955,  0.        , -0.05575981, ...,  0.        ,
         0.        ,  0.9       ]])